# Regresión Elastic Net: Teoría y Aplicación

Este notebook explora la regresión Elastic Net, combinando los beneficios de Ridge y Lasso, con teoría, implementación práctica y análisis estadístico del modelo.

## 1. ¿Por qué usar Elastic Net?

- Ridge: buena para colinealidad, pero no hace selección.
- Lasso: selecciona variables, pero puede ser inestable si hay variables altamente correlacionadas.
- **Elastic Net combina ambas**: selección y agrupamiento de coeficientes correlacionados.


## 2. Fundamento teórico
Elastic Net minimiza:
$$\text{min} \left\{ \| y - X\beta \|^2 + \lambda_1 \|\beta\|_1 + \lambda_2 \|\beta\|_2^2 \right\}$$

En `scikit-learn`, se parametriza como:
- `alpha`: controla la fuerza total de regularización
- `l1_ratio`: pondera entre Lasso (`1.0`) y Ridge (`0.0`)

Elastic Net tiende a:
- Seleccionar grupos de variables correlacionadas
- Mantener estabilidad frente a colinealidad

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

In [ ]:
# Preparar datos
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Ajustar Elastic Net
enet = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000).fit(X_train_scaled, y_train)
ols = LinearRegression().fit(X_train_scaled, y_train)

y_pred_enet = enet.predict(X_test_scaled)
y_pred_ols = ols.predict(X_test_scaled)

# Errores
print("Elastic Net:", mean_squared_error(y_test, y_pred_enet), r2_score(y_test, y_pred_enet))
print("OLS:", mean_squared_error(y_test, y_pred_ols), r2_score(y_test, y_pred_ols))

In [ ]:
# Visualización de coeficientes
plt.figure(figsize=(10, 5))
plt.plot(enet.coef_, label='Elastic Net')
plt.plot(ols.coef_, label='OLS')
plt.axhline(0, color='gray', linestyle='--')
plt.title("Coeficientes: Elastic Net vs OLS")
plt.legend()
plt.grid(True)
plt.show()

## 3. Análisis de residuos y supuestos
Evaluamos los errores del modelo Elastic Net para validar su consistencia estadística.

In [ ]:
# Análisis de residuos
residuals = y_test - y_pred_enet

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(residuals, kde=True, ax=ax[0])
sm.qqplot(residuals, line='s', ax=ax[1])
ax[0].set_title("Histograma de residuos")
ax[1].set_title("Q-Q plot")
plt.tight_layout()
plt.show()

# Shapiro-Wilk
stat, p = shapiro(residuals)
print("Shapiro-Wilk p-valor:", p)

## 4. Exploración de l1_ratio
Cómo cambian los coeficientes al variar la mezcla entre L1 y L2.

In [ ]:
l1_ratios = np.linspace(0.01, 1.0, 20)
coefs = []
for ratio in l1_ratios:
    en = ElasticNet(alpha=0.1, l1_ratio=ratio, max_iter=10000).fit(X_train_scaled, y_train)
    coefs.append(en.coef_)

plt.figure(figsize=(10, 6))
plt.plot(l1_ratios, coefs)
plt.xlabel("l1_ratio")
plt.ylabel("Coeficientes")
plt.title("Evolución de coeficientes en Elastic Net")
plt.grid(True)
plt.show()

## 5. Conclusión y aplicaciones
- Elastic Net combina selección de Lasso y estabilidad de Ridge
- Se adapta bien cuando hay muchas variables correlacionadas
- Muy usado en modelos genómicos, selección de atributos y análisis multivariante
- Permite control fino de la estructura del modelo mediante `l1_ratio` y `alpha`